In [5]:
import rdflib
import dotenv

import json
from rdflib import Graph, Namespace, URIRef, Literal, RDF, RDFS, OWL

# 1. Load your JSON and Ontology
with open('/Users/mc/Desktop/FD1/data_dictionaries_schema.json', 'r') as f:
    metadata = json.load(f)

g = Graph()
g.parse("/Users/mc/Downloads/data_democratization_ontology.owl", format="xml")

# 2. Define Namespaces (must match your .owl file)
EX = Namespace("http://example.org/fanduel/data-democratization-ontology#")
g.bind("ex", EX)

def map_metadata_to_owl():
    # Loop through tables in JSON
    for table in metadata['tables']:
        # Create a unique URI for the table
        # We sanitize the physical name to use as a fragment
        table_uri = EX[table['physical_name'].replace('.', '_')]
        
        # Assign Type: EnrichedTable or BaseTable
        table_type = EX.EnrichedTable if table['table_type'] == "ENRICHED" else EX.BaseTable
        g.add((table_uri, RDF.type, table_type))
        
        # Add metadata properties
        g.add((table_uri, RDFS.label, Literal(table['physical_name'])))
        g.add((table_uri, EX.assetName, Literal(table['physical_name'])))
        g.add((table_uri, EX.businessDefinition, Literal(table['description'])))

        # Map Columns
        for col in table['columns']:
            # Create unique Column URI (Table_Column)
            col_uri = EX[f"{table['physical_name'].replace('.', '_')}_{col['name']}"]
            g.add((col_uri, RDF.type, EX.Column))
            g.add((col_uri, EX.assetName, Literal(col['name'])))
            g.add((col_uri, EX.inferredDatatype, Literal(f"xsd:{col['data_type']}")))
            
            # Link Column to Table
            g.add((table_uri, EX.hasColumn, col_uri))
            g.add((col_uri, EX.belongsToTable, table_uri))

    # 3. Save the enriched ontology
    g.serialize(destination="dd_enriched_ontology.owl", format="xml")
    print("Mapping complete! Created dd_enriched_ontology.owl")

if __name__ == "__main__":
    map_metadata_to_owl()

http://example.org/fanduel/data-democratization-ontology#"foundation_views_financial" does not look like a valid URI, trying to serialize this will break.
http://example.org/fanduel/data-democratization-ontology#"foundation_views_financial"_deposits does not look like a valid URI, trying to serialize this will break.
http://example.org/fanduel/data-democratization-ontology#"foundation_views_financial"_deposits_additional_data does not look like a valid URI, trying to serialize this will break.
http://example.org/fanduel/data-democratization-ontology#"foundation_views_financial"_deposits_ani_first_name_response_code does not look like a valid URI, trying to serialize this will break.
http://example.org/fanduel/data-democratization-ontology#"foundation_views_financial"_deposits_ani_last_name_response_code does not look like a valid URI, trying to serialize this will break.
http://example.org/fanduel/data-democratization-ontology#"foundation_views_financial"_deposits_auth_reference does n

Mapping complete! Created dd_enriched_ontology.owl


In [7]:
import ollama
from rdflib import Graph, Namespace, Literal, RDF

# 1. Setup Environment
EX = Namespace("http://example.org/fanduel/data-democratization-ontology#")
g = Graph()
g.parse("dd_enriched_ontology.owl", format="xml")

def get_fan_duel_inference(asset_name, columns, existing_relations):
    """
    FanDuel-specific prompt to infer business logic from raw Databricks metadata.
    """
    prompt = f"""
    CONTEXT (FanDuel Data Lake):
    Technical Table: {asset_name}
    Technical Columns: {', '.join(columns)}
    Existing Ontological Links: {', '.join(existing_relations)}

    TASK:
    1. Define 'Purpose': Explain what this table does for a FanDuel Business Analyst (e.g., wallet management, fraud detection, marketing).
    2. 'Composable Join': Suggest which table this should join with to create a '360 View' of the customer (e.g., joining ledger lines with user profiles).
    3. 'Governance Risk': Rate 1-5 how sensitive this is for GDPR/AML compliance.
    4. 'Example Query': Write a simple SQL snippet that a user could copy/paste.

    FORMAT:
    Purpose: [text]
    JoinSuggestion: [Table Name]
    RiskScore: [1-5]
    SQLSnippet: [SQL code]
    """
    
    response = ollama.chat(model='llama3', messages=[{'role': 'user', 'content': prompt}])
    return response['message']['content']

def dd_enhance_fanduel_ontology():
    # Loop through tables identified in your OWL file
    for table_uri in g.subjects(RDF.type, None):
        if (table_uri, RDF.type, EX.EnrichedTable) in g or (table_uri, RDF.type, EX.BaseTable) in g:
            asset_name = str(g.value(table_uri, EX.assetName))
            # Extract technical columns to give Llama 3 context
            columns = [str(g.value(c, EX.assetName)) for c in g.objects(table_uri, EX.hasColumn)]
            # Extract existing relations (e.g., relatedToTable)
            relations = [str(o).split('#')[-1] for o in g.objects(table_uri, EX.relatedToTable)]
            
            print(f"Generating FanDuel Business Context for: {asset_name}...")
            ai_output = get_fan_duel_inference(asset_name, columns, relations)
            
            # Semantic Integration Logic
            lines = ai_output.split('\n')
            for line in lines:
                if line.startswith("Purpose:"):
                    g.set((table_uri, EX.businessDefinition, Literal(line.replace("Purpose:", "").strip())))
                if line.startswith("SQLSnippet:"):
                    g.set((table_uri, EX.exampleQuery, Literal(line.replace("SQLSnippet:", "").strip())))
                if line.startswith("RiskScore:"):
                    score = line.replace("RiskScore:", "").strip()
                    g.set((table_uri, EX.governanceScore, Literal(int(score) * 20))) # Scale 1-5 to 0-100

    g.serialize(destination="dd_smart_metadata.owl", format="xml")
    print("Success: dd_smart_metadata.owl created with Llama 3 intelligence.")

if __name__ == "__main__":
    dd_enhance_fanduel_ontology()

http://example.org/fanduel/data-democratization-ontology#"foundation_views_financial"_withdrawals_is_authorized_transition does not look like a valid URI, trying to serialize this will break.
http://example.org/fanduel/data-democratization-ontology#"foundation_views_financial"_withdrawals does not look like a valid URI, trying to serialize this will break.
http://example.org/fanduel/data-democratization-ontology#"foundation_views_financial"_withdrawals_refund_transfer_id does not look like a valid URI, trying to serialize this will break.
http://example.org/fanduel/data-democratization-ontology#"foundation_views_financial"_withdrawals does not look like a valid URI, trying to serialize this will break.
http://example.org/fanduel/data-democratization-ontology#"foundation_views_financial"_deposits_payment_reference does not look like a valid URI, trying to serialize this will break.
http://example.org/fanduel/data-democratization-ontology#"foundation_views_financial"_deposits does not lo

Generating FanDuel Business Context for: foundation.financial.withdrawals_enriched_v4...
Generating FanDuel Business Context for: "foundation_views.financial".ledger_lines_enriched...
Generating FanDuel Business Context for: foundation.financial.deposits_enriched_v1...
Generating FanDuel Business Context for: foundation.financial.ledger_lines_enriched_v1...
Generating FanDuel Business Context for: foundation.financial.ledger_lines_enriched...
Generating FanDuel Business Context for: "foundation_views.financial".worldpay_transactions...
Generating FanDuel Business Context for: "foundation_views.account".verification_attempt...
Generating FanDuel Business Context for: foundation.account.verified_user_details...
Generating FanDuel Business Context for: "foundation_views.financial".deposits...
Generating FanDuel Business Context for: "foundation.financial".ledger_account_balances_v1...
Generating FanDuel Business Context for: "foundation_views.account"...
Generating FanDuel Business Contex